In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import pyDOE

from torch.optim.optimizer import Optimizer, required

torch.manual_seed(16)
np.random.seed(16)
#random.seed(16)
torch.backends.cudnn.deterministic = True

In [2]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN,self).__init__()
        #layer definitions
        self.FC1 = nn.Linear(2,50)
        self.act1 = nn.Tanh()
        self.FC2 = nn.Linear(50,50)
        self.act2 = nn.Tanh()
        self.FC3 = nn.Linear(50,50)
        self.act3 = nn.Tanh()
        self.FC4 = nn.Linear(50,50)
        self.act4 = nn.Tanh()
        self.FC5 = nn.Linear(50,3)

    def forward(self, x, t):
        y  = torch.cat((x,t),1)
        x1 = self.FC1(y)
        x2 = self.act1(x1)
        x3 = self.FC2(x2)
        x4 = self.act2(x3)
        x5 = self.FC3(x4)
        x6 = self.act3(x5)
        x7 = self.FC4(x6)
        x8 = self.act4(x7)
        x9 = self.FC5(x8)
        return x9

In [3]:
"""
DE: uu_x + vu_y + p_x -(1/Re)(u_xx + u_yy) = 0    -0.5<= x<= 1  -0.5<= y<= 1.5
    uv_x + vv_y + p_y -(1/Re)(v_xx + v_yy) = 0
    u_x + v_y = 0
"""

Nf = 2600

#Residual
design = pyDOE.lhs(2, samples = Nf)
x = -0.5 + 1.5*design[:,0] #1st variable
y = -0.5 + 2*design[:,1] #2nd variable
f = np.zeros(Nf) #right hand side

# Generate samples for x, u [boundary conditions]
#boundary
x_b = np.linspace(-0.5,1,100)
y_b = np.linspace(-0.5,1.5,100)

y_0 = np.zeros(100)-0.5
y_1 = np.zeros(100)+1.5

x_0 = np.zeros(100)-0.5
x_1 = np.zeros(100)+1


#to tensor
x = x.reshape((-1, 1))
x = torch.tensor(x).type(torch.FloatTensor)
y = y.reshape((-1, 1))
y = torch.tensor(y).type(torch.FloatTensor)
f = f.reshape((-1, 1))
f = torch.tensor(f).type(torch.FloatTensor)


x_b = x_b.reshape((-1, 1))
x_b = torch.tensor(x_b).type(torch.FloatTensor)
x_0 = x_0.reshape((-1, 1))
x_0 = torch.tensor(x_0).type(torch.FloatTensor)
x_1 = x_1.reshape((-1, 1))
x_1 = torch.tensor(x_1).type(torch.FloatTensor)

y_b = y_b.reshape((-1, 1))
y_b = torch.tensor(y_b).type(torch.FloatTensor)
y_0 = y_0.reshape((-1, 1))
y_0 = torch.tensor(y_0).type(torch.FloatTensor)
y_1 = y_1.reshape((-1, 1))
y_1 = torch.tensor(y_1).type(torch.FloatTensor)

x_bc = torch.cat((x_b,x_b,x_0,x_1),0)
y_bc = torch.cat((y_0,y_1,y_b,y_b),0)

#u boundary
n = 1/40
l = 1/(2*n) - np.sqrt(1/(4*n**2) + 4*(np.pi)**2)

u_b = 1 - torch.exp(l*x_bc)*np.cos(2*np.pi*y_bc) 
v_b = (l/(2*np.pi))*torch.exp(l*x_bc)*np.sin(2*np.pi*y_bc) 

In [4]:
pn = PINN()

MAX_EPOCHS = 50000
LRATE = 1e-4

#Use Adam for training
optimizer = torch.optim.Adam(pn.parameters(), lr=LRATE)

criterion = nn.MSELoss()

loss_history_u = []
loss_history_f = []
loss_history = []

In [6]:
l=1
for epoch in range(MAX_EPOCHS):
    #full batch
    
    #u
    x_b = x_bc.clone()
    x_b.requires_grad = True

    y_b = y_bc.clone()
    y_b.requires_grad = True
    
    upred_bc = pn(x_b, y_b)
    
    u_bc = torch.reshape(upred_bc[:,0], (-1, 1))
    v_bc = torch.reshape(upred_bc[:,1], (-1, 1))

    mse_u = criterion(input=u_bc, target = u_b) + criterion(input=v_bc, target = v_b)

    loss_history_u.append([epoch, mse_u])

    #f
    xc = x.clone()
    xc.requires_grad = True

    yc = y.clone()
    yc.requires_grad = True

    upred = pn(xc, yc)
    
    u = torch.reshape(upred[:,0], (-1, 1))
    v = torch.reshape(upred[:,1], (-1, 1))
    p = torch.reshape(upred[:,2], (-1, 1))
    
    u_x = torch.autograd.grad(u.sum(),xc,create_graph=True)[0]
    v_x = torch.autograd.grad(v.sum(),xc,create_graph=True)[0]
    
    u_y = torch.autograd.grad(u.sum(),yc,create_graph=True)[0]
    v_y = torch.autograd.grad(v.sum(),yc,create_graph=True)[0]
    
    p_x = torch.autograd.grad(p.sum(),xc,create_graph=True)[0]
    p_y = torch.autograd.grad(p.sum(),yc,create_graph=True)[0]

    u_xx = torch.autograd.grad(u_x.sum(),xc,create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(),yc,create_graph=True)[0]
    
    v_xx = torch.autograd.grad(v_x.sum(),xc,create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y.sum(),yc,create_graph=True)[0]


    mse_f = criterion(input = u*u_x + v*u_y + p_x - (1/40)*(u_xx + u_yy), target = f )\
          + criterion(input = u*v_x + v*v_y + p_y - (1/40)*(v_xx + v_yy), target = f )\
          + criterion(input = u_x + v_y, target = f )
    

    loss_history_f.append([epoch, mse_f])
    
    
    loss = mse_u + mse_f
    loss_history.append([epoch, loss])
    
    #optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
        
    if (epoch+1) % 1 == 0:
        print("Epoch: {}, MSE_u: {:.7f}, MSE_f: {:.7f}, MSE: {:.7f}, l: {:.2f}".format((epoch+1), mse_u, mse_f, loss, l))

Epoch: 1, MSE_u: 2.3578429, MSE_f: 0.0121868, MSE: 2.3700297, l: 1.00
Epoch: 2, MSE_u: 2.3422167, MSE_f: 0.0118603, MSE: 2.3540771, l: 1.00
Epoch: 3, MSE_u: 2.3266799, MSE_f: 0.0115391, MSE: 2.3382192, l: 1.00
Epoch: 4, MSE_u: 2.3112288, MSE_f: 0.0112246, MSE: 2.3224533, l: 1.00
Epoch: 5, MSE_u: 2.2958624, MSE_f: 0.0109173, MSE: 2.3067796, l: 1.00
Epoch: 6, MSE_u: 2.2805769, MSE_f: 0.0106172, MSE: 2.2911942, l: 1.00
Epoch: 7, MSE_u: 2.2653711, MSE_f: 0.0103243, MSE: 2.2756953, l: 1.00
Epoch: 8, MSE_u: 2.2502418, MSE_f: 0.0100386, MSE: 2.2602804, l: 1.00
Epoch: 9, MSE_u: 2.2351875, MSE_f: 0.0097599, MSE: 2.2449474, l: 1.00
Epoch: 10, MSE_u: 2.2202048, MSE_f: 0.0094885, MSE: 2.2296934, l: 1.00
Epoch: 11, MSE_u: 2.2052927, MSE_f: 0.0092241, MSE: 2.2145169, l: 1.00
Epoch: 12, MSE_u: 2.1904480, MSE_f: 0.0089669, MSE: 2.1994150, l: 1.00
Epoch: 13, MSE_u: 2.1756692, MSE_f: 0.0087169, MSE: 2.1843860, l: 1.00
Epoch: 14, MSE_u: 2.1609521, MSE_f: 0.0084740, MSE: 2.1694262, l: 1.00
Epoch: 15, MSE_

Epoch: 117, MSE_u: 0.7950828, MSE_f: 0.0099454, MSE: 0.8050282, l: 1.00
Epoch: 118, MSE_u: 0.7877693, MSE_f: 0.0101351, MSE: 0.7979044, l: 1.00
Epoch: 119, MSE_u: 0.7806800, MSE_f: 0.0103246, MSE: 0.7910046, l: 1.00
Epoch: 120, MSE_u: 0.7738136, MSE_f: 0.0105139, MSE: 0.7843274, l: 1.00
Epoch: 121, MSE_u: 0.7671692, MSE_f: 0.0107027, MSE: 0.7778718, l: 1.00
Epoch: 122, MSE_u: 0.7607441, MSE_f: 0.0108908, MSE: 0.7716349, l: 1.00
Epoch: 123, MSE_u: 0.7545363, MSE_f: 0.0110781, MSE: 0.7656144, l: 1.00
Epoch: 124, MSE_u: 0.7485429, MSE_f: 0.0112644, MSE: 0.7598073, l: 1.00
Epoch: 125, MSE_u: 0.7427620, MSE_f: 0.0114495, MSE: 0.7542115, l: 1.00
Epoch: 126, MSE_u: 0.7371895, MSE_f: 0.0116332, MSE: 0.7488227, l: 1.00
Epoch: 127, MSE_u: 0.7318226, MSE_f: 0.0118154, MSE: 0.7436379, l: 1.00
Epoch: 128, MSE_u: 0.7266582, MSE_f: 0.0119958, MSE: 0.7386540, l: 1.00
Epoch: 129, MSE_u: 0.7216917, MSE_f: 0.0121743, MSE: 0.7338660, l: 1.00
Epoch: 130, MSE_u: 0.7169198, MSE_f: 0.0123508, MSE: 0.7292706, 

Epoch: 230, MSE_u: 0.6144769, MSE_f: 0.0144160, MSE: 0.6288929, l: 1.00
Epoch: 231, MSE_u: 0.6143719, MSE_f: 0.0143799, MSE: 0.6287518, l: 1.00
Epoch: 232, MSE_u: 0.6142692, MSE_f: 0.0143441, MSE: 0.6286133, l: 1.00
Epoch: 233, MSE_u: 0.6141686, MSE_f: 0.0143086, MSE: 0.6284772, l: 1.00
Epoch: 234, MSE_u: 0.6140699, MSE_f: 0.0142734, MSE: 0.6283433, l: 1.00
Epoch: 235, MSE_u: 0.6139733, MSE_f: 0.0142385, MSE: 0.6282118, l: 1.00
Epoch: 236, MSE_u: 0.6138786, MSE_f: 0.0142039, MSE: 0.6280825, l: 1.00
Epoch: 237, MSE_u: 0.6137856, MSE_f: 0.0141696, MSE: 0.6279551, l: 1.00
Epoch: 238, MSE_u: 0.6136944, MSE_f: 0.0141355, MSE: 0.6278299, l: 1.00
Epoch: 239, MSE_u: 0.6136050, MSE_f: 0.0141017, MSE: 0.6277066, l: 1.00
Epoch: 240, MSE_u: 0.6135170, MSE_f: 0.0140681, MSE: 0.6275852, l: 1.00
Epoch: 241, MSE_u: 0.6134306, MSE_f: 0.0140348, MSE: 0.6274655, l: 1.00
Epoch: 242, MSE_u: 0.6133460, MSE_f: 0.0140018, MSE: 0.6273478, l: 1.00
Epoch: 243, MSE_u: 0.6132625, MSE_f: 0.0139690, MSE: 0.6272315, 

Epoch: 345, MSE_u: 0.6075988, MSE_f: 0.0114259, MSE: 0.6190248, l: 1.00
Epoch: 346, MSE_u: 0.6075518, MSE_f: 0.0114073, MSE: 0.6189591, l: 1.00
Epoch: 347, MSE_u: 0.6075051, MSE_f: 0.0113887, MSE: 0.6188938, l: 1.00
Epoch: 348, MSE_u: 0.6074582, MSE_f: 0.0113702, MSE: 0.6188285, l: 1.00
Epoch: 349, MSE_u: 0.6074116, MSE_f: 0.0113518, MSE: 0.6187634, l: 1.00
Epoch: 350, MSE_u: 0.6073649, MSE_f: 0.0113336, MSE: 0.6186985, l: 1.00
Epoch: 351, MSE_u: 0.6073183, MSE_f: 0.0113154, MSE: 0.6186337, l: 1.00
Epoch: 352, MSE_u: 0.6072718, MSE_f: 0.0112973, MSE: 0.6185691, l: 1.00
Epoch: 353, MSE_u: 0.6072252, MSE_f: 0.0112793, MSE: 0.6185045, l: 1.00
Epoch: 354, MSE_u: 0.6071787, MSE_f: 0.0112614, MSE: 0.6184401, l: 1.00
Epoch: 355, MSE_u: 0.6071323, MSE_f: 0.0112436, MSE: 0.6183758, l: 1.00
Epoch: 356, MSE_u: 0.6070858, MSE_f: 0.0112258, MSE: 0.6183116, l: 1.00
Epoch: 357, MSE_u: 0.6070395, MSE_f: 0.0112082, MSE: 0.6182477, l: 1.00
Epoch: 358, MSE_u: 0.6069932, MSE_f: 0.0111907, MSE: 0.6181839, 

Epoch: 459, MSE_u: 0.6024988, MSE_f: 0.0097713, MSE: 0.6122701, l: 1.00
Epoch: 460, MSE_u: 0.6024557, MSE_f: 0.0097599, MSE: 0.6122155, l: 1.00
Epoch: 461, MSE_u: 0.6024128, MSE_f: 0.0097485, MSE: 0.6121613, l: 1.00
Epoch: 462, MSE_u: 0.6023699, MSE_f: 0.0097371, MSE: 0.6121070, l: 1.00
Epoch: 463, MSE_u: 0.6023270, MSE_f: 0.0097258, MSE: 0.6120529, l: 1.00
Epoch: 464, MSE_u: 0.6022841, MSE_f: 0.0097146, MSE: 0.6119987, l: 1.00
Epoch: 465, MSE_u: 0.6022415, MSE_f: 0.0097033, MSE: 0.6119448, l: 1.00
Epoch: 466, MSE_u: 0.6021986, MSE_f: 0.0096921, MSE: 0.6118907, l: 1.00
Epoch: 467, MSE_u: 0.6021557, MSE_f: 0.0096810, MSE: 0.6118367, l: 1.00
Epoch: 468, MSE_u: 0.6021131, MSE_f: 0.0096699, MSE: 0.6117829, l: 1.00
Epoch: 469, MSE_u: 0.6020703, MSE_f: 0.0096588, MSE: 0.6117290, l: 1.00
Epoch: 470, MSE_u: 0.6020277, MSE_f: 0.0096477, MSE: 0.6116754, l: 1.00
Epoch: 471, MSE_u: 0.6019849, MSE_f: 0.0096367, MSE: 0.6116216, l: 1.00
Epoch: 472, MSE_u: 0.6019421, MSE_f: 0.0096257, MSE: 0.6115679, 

...

Epoch: 3174, MSE_u: 0.1100182, MSE_f: 0.0276141, MSE: 0.1376322, l: 1.00
Epoch: 3175, MSE_u: 0.1092344, MSE_f: 0.0275371, MSE: 0.1367715, l: 1.00
Epoch: 3176, MSE_u: 0.1084296, MSE_f: 0.0274819, MSE: 0.1359115, l: 1.00
Epoch: 3177, MSE_u: 0.1076495, MSE_f: 0.0274029, MSE: 0.1350524, l: 1.00
Epoch: 3178, MSE_u: 0.1068442, MSE_f: 0.0273501, MSE: 0.1341943, l: 1.00
Epoch: 3179, MSE_u: 0.1060701, MSE_f: 0.0272671, MSE: 0.1333372, l: 1.00
Epoch: 3180, MSE_u: 0.1052627, MSE_f: 0.0272187, MSE: 0.1324814, l: 1.00
Epoch: 3181, MSE_u: 0.1044965, MSE_f: 0.0271302, MSE: 0.1316268, l: 1.00
Epoch: 3182, MSE_u: 0.1036845, MSE_f: 0.0270893, MSE: 0.1307738, l: 1.00
Epoch: 3183, MSE_u: 0.1029309, MSE_f: 0.0269917, MSE: 0.1299226, l: 1.00
Epoch: 3184, MSE_u: 0.1021087, MSE_f: 0.0269649, MSE: 0.1290736, l: 1.00
Epoch: 3185, MSE_u: 0.1013781, MSE_f: 0.0268495, MSE: 0.1282276, l: 1.00
Epoch: 3186, MSE_u: 0.1005330, MSE_f: 0.0268527, MSE: 0.1273858, l: 1.00
Epoch: 3187, MSE_u: 0.0998445, MSE_f: 0.0267054, MS

Epoch: 3287, MSE_u: 0.0422611, MSE_f: 0.0211767, MSE: 0.0634378, l: 1.00
Epoch: 3288, MSE_u: 0.0419198, MSE_f: 0.0211086, MSE: 0.0630284, l: 1.00
Epoch: 3289, MSE_u: 0.0415559, MSE_f: 0.0210664, MSE: 0.0626223, l: 1.00
Epoch: 3290, MSE_u: 0.0412266, MSE_f: 0.0209928, MSE: 0.0622195, l: 1.00
Epoch: 3291, MSE_u: 0.0408674, MSE_f: 0.0209523, MSE: 0.0618198, l: 1.00
Epoch: 3292, MSE_u: 0.0405462, MSE_f: 0.0208769, MSE: 0.0614232, l: 1.00
Epoch: 3293, MSE_u: 0.0401946, MSE_f: 0.0208351, MSE: 0.0610297, l: 1.00
Epoch: 3294, MSE_u: 0.0398792, MSE_f: 0.0207602, MSE: 0.0606394, l: 1.00
Epoch: 3295, MSE_u: 0.0395364, MSE_f: 0.0207160, MSE: 0.0602524, l: 1.00
Epoch: 3296, MSE_u: 0.0392261, MSE_f: 0.0206425, MSE: 0.0598685, l: 1.00
Epoch: 3297, MSE_u: 0.0388918, MSE_f: 0.0205961, MSE: 0.0594880, l: 1.00
Epoch: 3298, MSE_u: 0.0385869, MSE_f: 0.0205237, MSE: 0.0591106, l: 1.00
Epoch: 3299, MSE_u: 0.0382605, MSE_f: 0.0204759, MSE: 0.0587365, l: 1.00
Epoch: 3300, MSE_u: 0.0379616, MSE_f: 0.0204039, MS

Epoch: 3400, MSE_u: 0.0189675, MSE_f: 0.0149823, MSE: 0.0339498, l: 1.00
Epoch: 3401, MSE_u: 0.0188281, MSE_f: 0.0149679, MSE: 0.0337960, l: 1.00
Epoch: 3402, MSE_u: 0.0187427, MSE_f: 0.0149009, MSE: 0.0336436, l: 1.00
Epoch: 3403, MSE_u: 0.0186225, MSE_f: 0.0148733, MSE: 0.0334958, l: 1.00
Epoch: 3404, MSE_u: 0.0185255, MSE_f: 0.0148271, MSE: 0.0333526, l: 1.00
Epoch: 3405, MSE_u: 0.0184266, MSE_f: 0.0147851, MSE: 0.0332117, l: 1.00
Epoch: 3406, MSE_u: 0.0183196, MSE_f: 0.0147510, MSE: 0.0330706, l: 1.00
Epoch: 3407, MSE_u: 0.0182280, MSE_f: 0.0147003, MSE: 0.0329283, l: 1.00
Epoch: 3408, MSE_u: 0.0181194, MSE_f: 0.0146666, MSE: 0.0327861, l: 1.00
Epoch: 3409, MSE_u: 0.0180272, MSE_f: 0.0146185, MSE: 0.0326457, l: 1.00
Epoch: 3410, MSE_u: 0.0179263, MSE_f: 0.0145822, MSE: 0.0325085, l: 1.00
Epoch: 3411, MSE_u: 0.0178328, MSE_f: 0.0145412, MSE: 0.0323740, l: 1.00
Epoch: 3412, MSE_u: 0.0177388, MSE_f: 0.0145023, MSE: 0.0322411, l: 1.00
Epoch: 3413, MSE_u: 0.0176447, MSE_f: 0.0144639, MS

Epoch: 3513, MSE_u: 0.0118030, MSE_f: 0.0114768, MSE: 0.0232798, l: 1.00
Epoch: 3514, MSE_u: 0.0117727, MSE_f: 0.0114619, MSE: 0.0232346, l: 1.00
Epoch: 3515, MSE_u: 0.0117433, MSE_f: 0.0114472, MSE: 0.0231905, l: 1.00
Epoch: 3516, MSE_u: 0.0117129, MSE_f: 0.0114307, MSE: 0.0231436, l: 1.00
Epoch: 3517, MSE_u: 0.0116797, MSE_f: 0.0114069, MSE: 0.0230867, l: 1.00
Epoch: 3518, MSE_u: 0.0116385, MSE_f: 0.0113784, MSE: 0.0230170, l: 1.00
Epoch: 3519, MSE_u: 0.0116000, MSE_f: 0.0113382, MSE: 0.0229383, l: 1.00
Epoch: 3520, MSE_u: 0.0115532, MSE_f: 0.0113097, MSE: 0.0228629, l: 1.00
Epoch: 3521, MSE_u: 0.0115244, MSE_f: 0.0112766, MSE: 0.0228010, l: 1.00
Epoch: 3522, MSE_u: 0.0114879, MSE_f: 0.0112653, MSE: 0.0227532, l: 1.00
Epoch: 3523, MSE_u: 0.0114674, MSE_f: 0.0112444, MSE: 0.0227118, l: 1.00
Epoch: 3524, MSE_u: 0.0114335, MSE_f: 0.0112328, MSE: 0.0226663, l: 1.00
Epoch: 3525, MSE_u: 0.0114045, MSE_f: 0.0112066, MSE: 0.0226111, l: 1.00
Epoch: 3526, MSE_u: 0.0113666, MSE_f: 0.0111814, MS

KeyboardInterrupt: 

In [7]:
%matplotlib qt

In [14]:
X = np.linspace(-0.5,1,1000)
Y = np.linspace(-0.5,1.5,1000)
X, Y = np.meshgrid(X, Y)

#analytic solution
n = 1/40
l = 1/(2*n) - np.sqrt(1/(4*n**2) + 4*(np.pi)**2)
#U =  (l/(2*np.pi))*np.exp(l*X)*np.sin(2*np.pi*Y) 
U = 0.5*(1 - np.exp(2*l*X))
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_wireframe(X, Y, U)

In [15]:
X = np.linspace(-0.5,1,1000)
Y = np.linspace(-0.5,1.5,1000)
X, Y = np.meshgrid(X, Y)

x = X.reshape((-1, 1))
x = torch.tensor(x).type(torch.FloatTensor)

y = Y.reshape((-1, 1))
y = torch.tensor(y).type(torch.FloatTensor)

V = pn(x,y)

In [16]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_wireframe(X, Y,V[:,2].detach().numpy().reshape(1000,1000))
ax.plot_wireframe(X, Y, U, color='r')

In [17]:
torch.cuda.is_available()

False